In [1]:
import pandas as pd
import numpy as np
from script import *
import itertools
import lightgbm as lgbm
import matplotlib.pyplot as plt

import time

In [2]:
pd.set_option('display.max_rows',200)
idx = pd.IndexSlice
agg_operations =['mean','median','max','min','std','size']

In [6]:
start = 20100101
cursor = dbop.connect_db("sqlite3").cursor()

df_d = dbop.create_df(cursor, STOCK_DAY[TABLE], start=start)
df_d = dp.proc_stock_d(dp.prepare_stock_d(df_d))
df_d.drop(columns=['open0','high0','low0','vol0'],inplace=True)
df_d['pct'] = df_d.sort_index().groupby('code')['close'].pct_change()*100
df_d['amt']/=1e5
df_d = df_d.astype('float32')
print(df_d.columns)
print(df_d.shape)
df_d.info(memory_usage='deep')

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct'],
      dtype='object')
(3064850, 11)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 3064850 entries, (000001.SZ, 2010-01-04 00:00:00) to (603997.SH, 2020-04-09 00:00:00)
Data columns (total 11 columns):
open          float32
high          float32
low           float32
close         float32
vol           float32
amt           float32
adj_factor    float32
avg           float32
close0        float32
avg0          float32
pct           float32
dtypes: float32(11)
memory usage: 220.4 MB


In [7]:
# df_d_basic = dbop.create_df(cursor, STOCK_DAILY_BASIC[TABLE], start=start)
# df_d_basic = dp.prepare_stock_d_basic(df_d_basic)
# #
# df_d_basic["pb*pe_ttm"] = df_d_basic["pb"] * df_d_basic["pe_ttm"]
# df_d_basic["pb*pe"] = df_d_basic["pb"] * df_d_basic["pe"]

df_d_basic = df_d_basic.astype('float32')
print(df_d_basic.columns)
print(df_d_basic.shape)
df_d_basic.info(memory_usage='deep')

Index(['close', 'turnover_rate', 'turnover_rate_f', 'volume_ratio', 'pe',
       'pe_ttm', 'pb', 'ps', 'ps_ttm', 'total_share', 'float_share',
       'free_share', 'total_mv', 'circ_mv', 'pb*pe_ttm', 'pb*pe'],
      dtype='object')
(2904815, 16)
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 2904815 entries, (000001.SZ, 2010-01-04 00:00:00) to (603997.SH, 2020-04-09 00:00:00)
Data columns (total 16 columns):
close              float32
turnover_rate      float32
turnover_rate_f    float32
volume_ratio       float32
pe                 float32
pe_ttm             float32
pb                 float32
ps                 float32
ps_ttm             float32
total_share        float32
float_share        float32
free_share         float32
total_mv           float32
circ_mv            float32
pb*pe_ttm          float32
pb*pe              float32
dtypes: float32(16)
memory usage: 188.5 MB


In [8]:
df = df_d
periods = [5,10,20,30,60,120,250]
for k in periods:
    df['{:d}ma'.format(k)]=df.reset_index('code').groupby('code')['close'].rolling(k).mean()
print(df.columns)
    
for col2 in ["{}ma".format(k) for k in periods]:
    col1 = 'close'
    df['{}/{}'.format(col1,col2)] = df[col1]/df[col2]
print(df.columns)

Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma'],
      dtype='object')
Index(['open', 'high', 'low', 'close', 'vol', 'amt', 'adj_factor', 'avg',
       'close0', 'avg0', 'pct', '5ma', '10ma', '20ma', '30ma', '60ma', '120ma',
       '250ma', 'close/5ma', 'close/10ma', 'close/20ma', 'close/30ma',
       'close/60ma', 'close/120ma', 'close/250ma'],
      dtype='object')


In [10]:
# df = df.join(df_d_basic.drop(columns='close'),how='inner')

(2902921, 40)


In [16]:
# index = df_d.index[(df_d['close/120ma']<=0.69)
#                  & (df_d['close/30ma']<=0.778)
#                  & (df_d['close/60ma']<=0.65)
#                  & (df_d['close/60ma']>=0.55)
#                 ]
# print(df['turnover_rate_f'].head())
# df['p10mean_turnover_rate_f'] = df.reset_index('code').groupby('code')['turnover_rate_f'].rolling(10).mean()
stock_mask = (df['circ_mv']>50e4) & (df['total_mv']>100e4) & (df['close']>5) & (df['p10mean_turnover_rate_f']<5)
print(stock_mask.sum(),stock_mask.mean())
# print(len(index))

1080256 0.372127247003966


In [17]:
start_date = '-'.join([str(start)[:4],str(start)[4:6],str(start)[6:8]])
df_r = pd.read_parquet(r"database\return_5%_10%_20_8_15%").loc[idx[:,start_date:],:].sort_index()

In [ ]:
ml.assess